# Final Output Notebook

## Imports and Loading Data

In [1]:
# Imports
import pandas as pd
import numpy as np
import project_functions as pf
import project_models as pm
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

2022-04-27 11:21:02.277569: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Plotting config
pf.set_plot_config()
# Hide TF messages for neater NB
pf.tf_message_toggle("3")
# Set global seed for reproducible notebook
pf.set_seed()

In [3]:
# Hyperparams used during modelling
# Compilation hyperparams
compile_hp = dict()
compile_hp["loss"] = "binary_crossentropy"
compile_hp["optimizer"] = optimizers.Adam(learning_rate = 0.001)
compile_hp["metrics"] = ["accuracy"]

# Fitting hyperparams
fit_hp = dict()
fit_hp["batch_size"] = 64
fit_hp["epochs"] = 100
fit_hp["validation_split"] = 0.3
# Create callback to select the best model
fit_hp["callbacks"] = EarlyStopping(monitor = "val_loss",
                                         mode = "min",
                                         restore_best_weights = True,
                                         patience = 25)

# Eliminate verbose to have a neater notebook 
fit_hp["verbose"] = 0

## Pre-processing

In [4]:
# Load in data
text = pf.imdb_train_test_dfs()

In [5]:
# Split into test and train data
X = text.review.values
y = np.array(text["sentiment"], dtype = "float32")
text_train, text_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2, shuffle = True,
                                                          # reproducible split
                                                         random_state = 1)

In [6]:
# Process (lower-case, no special chars etc), tokenize, pad and convert to sequences
# Uses pre-padding and post-truncating with a max sequence length of 500
tokenizer, X_train, X_test = pf.tokenize_padder(text_train, text_test)

# Get vocab size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
# Toggle notebook options
display = False
save_plots = False
save_model = False
save_history = False

# Create dataframe to store each model's results on test set
all_results = pd.DataFrame()

## Create Embedding Layers

In [8]:
# Store in dictionary to make selection more programmatic throughout NB
# This will take a  while
embed_dict = dict()
embed_dict[None] = None
embed_dict["glove"] = pm.glove_embedding(tokenizer, show_progress = False)
# Takes the longest
embed_dict["spacy"] = pm.spacy_embedding(tokenizer, show_progress = True)
embed_dict["keras"] = pm.keras_embedding(tokenizer)

8.875 % complete
17.749 % complete
26.624 % complete
35.499 % complete
44.373 % complete
53.248 % complete
62.123 % complete
70.998 % complete
79.872 % complete
88.747 % complete
97.622 % complete


# MLP

## No Embedding

In [9]:
# Set model architecture
model_dict = dict()
model_dict["name"] = "mlp"
model_dict["embedding"] = None

### No Regularisation or Batch Normalisation

In [ ]:
pf.set_seed()
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

### Regularisation and No Batch Normalisation

In [11]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Linear_Dense_Elasti (Dense)  (None, 256)               128256    
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 128,513
Trainable params: 128,513
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [12]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Linear_Dense (Dense)         (None, 256)               128256    
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 129,537
Trainable params: 129,025
Non-trainable params: 512
_________________________________________________________________


### Both Regularisation and Batch Normalisation

In [13]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Linear_Dense_Elasti (Dense)  (None, 256)               128256    
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 129,537
Trainable params: 129,025
Non-trainable params: 512
_________________________________________________________________


## GloVe Embedding

In [14]:
# Set model architecture
model_dict["embedding"] = "glove"

### No Regularisation or Batch Normalisation

In [15]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
flatten (Flatten)            (None, 150000)            0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               38400256  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 72,204,513
Trainable params: 38,400,513
Non-trainable params: 33,804,000
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [16]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
flatten_1 (Flatten)          (None, 150000)            0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               38400256  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 72,204,513
Trainable params: 38,400,513
Non-trainable params: 33,804,000
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [17]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
flatten_2 (Flatten)          (None, 150000)            0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               38400256  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 72,205,537
Trainable params: 38,401,025
Non-trainable params: 33,804,512
__________________________________________

### Both Regularisation and Batch Normalisation

In [18]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
flatten_3 (Flatten)          (None, 150000)            0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               38400256  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 72,205,537
Trainable params: 38,401,025
Non-trainable params: 33,804,512
__________________________________________

## SpaCy Embedding

In [19]:
# Set model architecture
model_dict["embedding"] = "spacy"

### No Regularisation or Batch Normalisation

In [20]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
flatten_4 (Flatten)          (None, 48000)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               12288256  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 23,105,793
Trainable params: 12,288,513
Non-trainable params: 10,817,280
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [21]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
flatten_5 (Flatten)          (None, 48000)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               12288256  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 23,105,793
Trainable params: 12,288,513
Non-trainable params: 10,817,280
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [22]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
flatten_6 (Flatten)          (None, 48000)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               12288256  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 23,106,817
Trainable params: 12,289,025
Non-trainable params: 10,817,792
__________________________________________

### Both Regularisation and Batch Normalisation

In [23]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
flatten_7 (Flatten)          (None, 48000)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               12288256  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 23,106,817
Trainable params: 12,289,025
Non-trainable params: 10,817,792
__________________________________________

## Keras Embedding

In [24]:
# Set model architecture
model_dict["embedding"] = "keras"

### No Regularisation or Batch Normalisation

In [25]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
flatten_8 (Flatten)          (None, 128000)            0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               32768256  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 61,614,593
Trainable params: 61,614,593
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [26]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
flatten_9 (Flatten)          (None, 128000)            0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               32768256  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 61,614,593
Trainable params: 61,614,593
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [27]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
flatten_10 (Flatten)         (None, 128000)            0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               32768256  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 61,615,617
Trainable params: 61,615,105
Non-trainable params: 512
_________________________________________________

### Both Regularisation and Batch Normalisation

In [28]:

# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.mlp(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
flatten_11 (Flatten)         (None, 128000)            0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               32768256  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 61,615,617
Trainable params: 61,615,105
Non-trainable params: 512
_________________________________________________

- Save results from MLPs

In [29]:
all_results.to_csv("all_results_table.csv", index = False)

# CNN

## No Embedding

In [30]:
# Set model architecture
model_dict = dict()
model_dict["name"] = "cnn"
model_dict["embedding"] = None

### No Regularisation or Batch Normalisation

In [31]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          2304      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 246, 256)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

2022-04-27 11:34:09.594774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-04-27 11:34:10.850932: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-04-27 11:34:10.940601: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


### Regularisation and No Batch Normalisation

In [32]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          2304      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Batch Normalisation and No Regularisation

In [33]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          2304      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

### Both Regularisation and Batch Normalisation

In [34]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          2304      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

## GloVe Embedding

In [35]:
# Set model architecture
model_dict["embedding"] = "glove"

### No Regularisation or Batch Normalisation

In [36]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          614656    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Regularisation and No Batch Normalisation

In [37]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          614656    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Batch Normalisation and No Regularisation

In [38]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          614656    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

### Both Regularisation and Batch Normalisation

In [39]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          614656    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

## SpaCy Embedding

In [40]:
# Set model architecture
model_dict["embedding"] = "spacy"

### No Regularisation or Batch Normalisation

In [41]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          196864    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Regularisation and No Batch Normalisation

In [42]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          196864    
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 246, 256)          0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Batch Normalisation and No Regularisation

In [43]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          196864    
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 246, 256)          0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

### Both Regularisation and Batch Normalisation

In [44]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          196864    
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 246, 256)          0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

## Keras Embedding

In [45]:
# Set model architecture
model_dict["embedding"] = "keras"

### No Regularisation or Batch Normalisation

In [46]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          524544    
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 246, 256)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Regularisation and No Batch Normalisation

In [47]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          524544    
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 246, 256)          0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257     

### Batch Normalisation and No Regularisation

In [48]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          524544    
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 246, 256)          0         
_________________________________________________________________
flatten_26 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

### Both Regularisation and Batch Normalisation

In [49]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.cnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Conv_layer (Conv1D)          (None, 493, 256)          524544    
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 246, 256)          0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 62976)             0         
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16122112  
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0       

- Save results from CNNs

In [50]:
all_results.to_csv("all_results_table.csv", index = False)

# RNN

## No Embedding

In [51]:
# Set model architecture
model_dict = dict()
model_dict["name"] = "rnn"
model_dict["embedding"] = None

### No Regularisation or Batch Normalisation

In [52]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_RNN (Bidirecti (None, 64)                2176      
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 19,073
Trainable params: 19,073
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [53]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_RNN (Bidirecti (None, 64)                2176      
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 19,073
Trainable params: 19,073
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [54]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_RNN (Bidirecti (None, 64)                2176      
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 20,097
Trainable params: 19,585
Non-trainable params: 512
_________________________________________________________________


### Both Regularisation and Batch Normalisation

In [55]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_RNN (Bidirecti (None, 64)                2176      
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 20,097
Trainable params: 19,585
Non-trainable params: 512
_________________________________________________________________


## GloVe Embedding

In [56]:
# Set model architecture
model_dict["embedding"] = "glove"

### No Regularisation or Batch Normalisation

In [57]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                21312     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,842,209
Trainable params: 38,209
Non-trainable params: 33,804,000
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [58]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                21312     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,842,209
Trainable params: 38,209
Non-trainable params: 33,804,000
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [59]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                21312     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,843,233
Trainable params: 38,721
Non-trainable params: 33,804,512
______________________________________________

### Both Regularisation and Batch Normalisation

In [60]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                21312     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,843,233
Trainable params: 38,721
Non-trainable params: 33,804,512
______________________________________________

## SpaCy Embedding

In [61]:
# Set model architecture
model_dict["embedding"] = "spacy"

### No Regularisation or Batch Normalisation

In [62]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                8256      
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,842,433
Trainable params: 25,153
Non-trainable params: 10,817,280
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [63]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                8256      
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,842,433
Trainable params: 25,153
Non-trainable params: 10,817,280
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [64]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                8256      
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,843,457
Trainable params: 25,665
Non-trainable params: 10,817,792
______________________________________________

### Both Regularisation and Batch Normalisation

In [65]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                8256      
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,843,457
Trainable params: 25,665
Non-trainable params: 10,817,792
______________________________________________

## Keras Embedding

In [66]:
# Set model architecture
model_dict["embedding"] = "keras"

### No Regularisation or Batch Normalisation

In [67]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                18496     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,881,473
Trainable params: 28,881,473
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [68]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                18496     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,881,473
Trainable params: 28,881,473
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [69]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                18496     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,882,497
Trainable params: 28,881,985
Non-trainable params: 512
_________________________________________________

### Both Regularisation and Batch Normalisation

In [70]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_rnn(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_RNN (Bidirecti (None, 64)                18496     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,882,497
Trainable params: 28,881,985
Non-trainable params: 512
_________________________________________________

- Save results from RNNs

In [71]:
all_results.to_csv("all_results_table.csv", index = False)

# LSTM

## No Embedding

In [72]:
# Set model architecture
model_dict = dict()
model_dict["name"] = "lstm"
model_dict["embedding"] = None

### No Regularisation or Batch Normalisation

In [75]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_LSTM (Bidirect (None, 64)                8704      
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 25,601
Trainable params: 25,601
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [79]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_LSTM (Bidirect (None, 64)                8704      
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 25,601
Trainable params: 25,601
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [80]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_LSTM (Bidirect (None, 64)                8704      
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 26,625
Trainable params: 26,113
Non-trainable params: 512
_________________________________________________________________


### Both Regularisation and Batch Normalisation

In [81]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Bidirectional_LSTM (Bidirect (None, 64)                8704      
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 26,625
Trainable params: 26,113
Non-trainable params: 512
_________________________________________________________________


## GloVe Embedding

In [82]:
# Set model architecture
model_dict["embedding"] = "glove"

### No Regularisation or Batch Normalisation

In [83]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                85248     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,906,145
Trainable params: 102,145
Non-trainable params: 33,804,000
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [84]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                85248     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,906,145
Trainable params: 102,145
Non-trainable params: 33,804,000
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [85]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                85248     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,907,169
Trainable params: 102,657
Non-trainable params: 33,804,512
____________________________________________

### Both Regularisation and Batch Normalisation

In [86]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                85248     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,907,169
Trainable params: 102,657
Non-trainable params: 33,804,512
____________________________________________

## SpaCy Embedding

In [87]:
# Set model architecture
model_dict["embedding"] = "spacy"

### No Regularisation or Batch Normalisation

In [88]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                33024     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,867,201
Trainable params: 49,921
Non-trainable params: 10,817,280
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [89]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                33024     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,867,201
Trainable params: 49,921
Non-trainable params: 10,817,280
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [90]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                33024     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,868,225
Trainable params: 50,433
Non-trainable params: 10,817,792
_____________________________________________

### Both Regularisation and Batch Normalisation

In [91]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                33024     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,868,225
Trainable params: 50,433
Non-trainable params: 10,817,792
_____________________________________________

## Keras Embedding

In [92]:
# Set model architecture
model_dict["embedding"] = "keras"

### No Regularisation or Batch Normalisation

In [93]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                73984     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,936,961
Trainable params: 28,936,961
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [94]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                73984     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,936,961
Trainable params: 28,936,961
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [95]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                73984     
_________________________________________________________________
Hidden_dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,937,985
Trainable params: 28,937,473
Non-trainable params: 512
________________________________________________

### Both Regularisation and Batch Normalisation

In [96]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_lstm(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_LSTM (Bidirect (None, 64)                73984     
_________________________________________________________________
Regularized_hidden_dense (De (None, 256)               16640     
_________________________________________________________________
Batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
ReLu_activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,937,985
Trainable params: 28,937,473
Non-trainable params: 512
________________________________________________

- Save results from LSTMs

In [97]:
all_results.to_csv("all_results_table.csv", index = False)

# GRU

## No Embedding

In [98]:
# Set model architecture
model_dict = dict()
model_dict["name"] = "gru"
model_dict["embedding"] = None

### No Regularisation or Batch Normalisation

In [99]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                6720      
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 23,617
Trainable params: 23,617
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [100]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                6720      
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 23,617
Trainable params: 23,617
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [101]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                6720      
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 24,641
Trainable params: 24,129
Non-trainable params: 512
_________________________________________________________

### Both Regularisation and Batch Normalisation

In [102]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Reshaping (Reshape)          (None, 500, 1)            0         
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                6720      
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 24,641
Trainable params: 24,129
Non-trainable params: 512
_________________________________________________________

## GloVe Embedding

In [103]:
# Set model architecture
model_dict["embedding"] = "glove"

### No Regularisation or Batch Normalisation

In [104]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                64128     
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,885,025
Trainable params: 81,025
Non-trainable params: 33,804,000
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [105]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                64128     
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,885,025
Trainable params: 81,025
Non-trainable params: 33,804,000
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [106]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                64128     
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,886,049
Trainable params: 81,537
Non-trainable params: 33,804,512
______________________________________________

### Both Regularisation and Batch Normalisation

In [107]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
glove_embedding (Embedding)  (None, 500, 300)          33804000  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                64128     
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 33,886,049
Trainable params: 81,537
Non-trainable params: 33,804,512
______________________________________________

## SpaCy Embedding

In [108]:
# Set model architecture
model_dict["embedding"] = "spacy"

### No Regularisation or Batch Normalisation

In [109]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                24960     
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,859,137
Trainable params: 41,857
Non-trainable params: 10,817,280
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [110]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                24960     
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,859,137
Trainable params: 41,857
Non-trainable params: 10,817,280
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [111]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                24960     
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,860,161
Trainable params: 42,369
Non-trainable params: 10,817,792
______________________________________________

### Both Regularisation and Batch Normalisation

In [112]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spacy_embedding (Embedding)  (None, 500, 96)           10817280  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                24960     
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 10,860,161
Trainable params: 42,369
Non-trainable params: 10,817,792
______________________________________________

## Keras Embedding

In [113]:
# Set model architecture
model_dict["embedding"] = "keras"

### No Regularisation or Batch Normalisation

In [114]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                55680     
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,918,657
Trainable params: 28,918,657
Non-trainable params: 0
_________________________________________________________________


### Regularisation and No Batch Normalisation

In [115]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = False

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                55680     
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,918,657
Trainable params: 28,918,657
Non-trainable params: 0
_________________________________________________________________


### Batch Normalisation and No Regularisation

In [116]:
# Tweak model architecture
model_dict["regularize"] = False
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                55680     
_________________________________________________________________
Linear_Dense (Dense)         (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,919,681
Trainable params: 28,919,169
Non-trainable params: 512
_________________________________________________

### Both Regularisation and Batch Normalisation

In [117]:
# Tweak model architecture
model_dict["regularize"] = True
model_dict["batch_normalize"] = True

# Build and compile
model = pm.bi_gru(regularize = model_dict["regularize"],
               batch_normalize = model_dict["batch_normalize"],
               embedding = embed_dict[model_dict["embedding"]],
               **compile_hp)
model.summary()
# Fit model and save it and its history
history = pm.fit_and_save(X_train, y_train, model,
                          save_model = save_model, 
                          save_history = save_history,
                          **model_dict, **fit_hp)
# Evaluate on test data
all_results = pm.get_test_metrics(model, X_test, y_test, all_results, history, **model_dict)
# Plot and save graphs
pf.plot_and_save(history, save = save_plots, display = display, **model_dict)

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_embedding (Embedding)  (None, 500, 256)          28846080  
_________________________________________________________________
Bidirectional_GRU (Bidirecti (None, 64)                55680     
_________________________________________________________________
Linear_Dense_Elasti (Dense)  (None, 256)               16640     
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 256)               1024      
_________________________________________________________________
ReLU_Activation (Activation) (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 1)                 257       
Total params: 28,919,681
Trainable params: 28,919,169
Non-trainable params: 512
_________________________________________________

- Save results from GRUs

In [118]:
all_results.to_csv("all_results_table.csv", index = False)